In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(filepath_or_buffer='/kaggle/input/top-100-billboard/billboard.csv', parse_dates=['week_id'])
df['score'] = 101 - df['week_position']
df['previous_score'] = df['previous_week_position'].apply(func=lambda x: np.nan if np.isnan(x) else 101 - x)
df['best_score'] = 101 - df['peak_position']
df['year'] = df['week_id'].dt.year
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
from plotly.express import histogram
histogram(data_frame=df, x='peak_position')

In [ ]:
df['peak_position'].value_counts(normalize=True).nlargest(n=1)

Is it a surprise that fully 5% of our data is for songs that reach the top spot? It is to me. It also suggests that the graph above doesn't measure what we want it to, which is the percentage of songs that reach the top spot.

In [ ]:
df[['song', 'performer', 'peak_position']].drop_duplicates(ignore_index=True).shape

Let's try getting the unique songs by dropping duplicates of song/performer (because multiple songs can have the same title) and then remove duplicate records for that song by having one record per peak position.

In [ ]:
histogram(data_frame=df[['song', 'performer', 'peak_position']].drop_duplicates(ignore_index=True), x='peak_position')

This makes more sense: number one records are rare. How rare are they?

In [ ]:
from plotly.express import scatter

peaks_df = df[['song', 'performer', 'peak_position']].drop_duplicates(ignore_index=True)['peak_position'].value_counts(normalize=True).to_frame().reset_index()
scatter(data_frame=peaks_df, x='peak_position', y='proportion')

Weirdly there are more songs that reach the top spot than reach the next-best spot. And there's something weird going on with positions 91+.

In [ ]:
columns = ['score', 'previous_score', 'best_score', 'weeks_on_chart', 'song', 'performer', 'week_id']
not_null_df = df[columns].dropna()
not_null_df.shape

For the following plots we are only looking at songs already in the chart, i.e. no weeks where the previous score is nan.

In [ ]:
scatter(data_frame=not_null_df.sample(n=10000), x='score', y='previous_score', color='best_score',
        hover_name='song', hover_data=['performer', 'week_id'])

Converting the positions to scores just gives bigger numbers rather than smaller numbers to the top songs, so our scatter now fits our intuition with better chart performance in the top right rather than bottom left corner.

In [ ]:
from random import sample
songs = sample(population=not_null_df['song'].unique().tolist(), k=1000)
songs_df = not_null_df[not_null_df['song'].isin(songs)]
print(songs_df.shape)
scatter(data_frame=songs_df, x='score', y='previous_score', color='best_score', hover_name='song', hover_data=['performer', 'week_id'])

In [ ]:
performers = sample(population=not_null_df['performer'].unique().tolist(), k=1000)
performers_df = not_null_df[not_null_df['performer'].isin(performers)]
print(performers_df.shape)
scatter(data_frame=performers_df, x='score', y='previous_score', color='best_score', hover_name='song', hover_data=['performer', 'week_id'])

In [ ]:
from plotly.express import bar
bar(data_frame=df['performer'].value_counts().nlargest(n=20).to_frame().reset_index(), x='performer', y='count')

These are the artists with the most song/weeks in the dataset. We can with a little work see who had the best year(s).

In [ ]:
bar(data_frame=df[['performer', 'year']].value_counts().nlargest(n=50).to_frame().reset_index(), x='performer', y='count', color='year')

Because each performer takes up linear space we can't put a lot of data in this chart, and we probably end up inviting comparisons between artists that are not reasonable. Maybe this data would look better as a scatter plot.

In [ ]:
scatter(data_frame=df[['performer', 'year']].value_counts().nlargest(n=1000).to_frame().reset_index(), x='year', y='count', color='year',
       hover_name='performer')

We can fit a lot more data into this chart, probably to the point of diminishing returns. This gives us a sense of what chart domination looks like on an annual-bucket basis. I'm not sure using the year for both the x dimension and the color is buying us much here, but we only have three data dimensions to plot and one of those is categorical.

In [ ]:
scatter(data_frame=df[df['performer'].isin({'The Beatles', 'Wings', 'John Lennon', 'George Harrison', 'Ringo Starr',
                                           'Paul McCartney'})], x='year', y='score', color='best_score', hover_name='song', hover_data=['performer','weeks_on_chart'])

In [ ]:
scatter(data_frame=df[df['performer'].isin({'The Beatles', 'Wings', 'John Lennon', 'George Harrison', 'Ringo Starr',
                                           'Paul McCartney'})], x='year', y='score', color='performer', hover_name='song', hover_data=['performer','weeks_on_chart'])

In [ ]:
scatter(data_frame=df[df['performer'] == 'Taylor Swift'], x='year', y='score', color='best_score', hover_name='song', hover_data=['weeks_on_chart'])

In [ ]:
scatter(data_frame=df[df['performer'].isin({'Nirvana', 'Pearl Jam', 'Radiohead'})], x='year', y='score', color='best_score', hover_name='song',
       hover_data=['performer', 'weeks_on_chart'])

We may have some data quality issues with our chart peak data; some of the Nirvana 1991 records have multiple values for best score.

In [ ]:
scatter(data_frame=df[df['performer'].isin({'Drake', 'Kanye West'})], x='year', y='score', color='best_score', hover_name='song',
       hover_data=['performer', 'weeks_on_chart'])